# Prepare the data

In [ ]:
provinces = ["Long An", "Hà Nội", "Hồ Chí Minh", "Bắc Ninh"]
districts = ["Cần Giuộc", "Quận 1"]
wards = ["Thuận Thành", "An Phú"]

# Normalized functions
https://github.com/phuochtran/address-classification/blob/main/gpt.md

In [ ]:
import re
import unicodedata
import time

def strip_accents(s: str) -> str:
    s = unicodedata.normalize('NFD', s)
    s = ''.join(ch for ch in s if unicodedata.category(ch) != 'Mn')
    return unicodedata.normalize('NFC', s)

def normalize(s: str) -> str:
    s = s.lower()
    # replace common punctuation with spaces
    s = re.sub(r'[\.,;/\\\(\)\[\]\-_:]', ' ', s)
    s = s.replace('\'', ' ')
    s = re.sub(r'\s+', ' ', s).strip()
    s = strip_accents(s)
    # remove any non-alphanumeric (keep spaces)
    s = re.sub(r'[^a-z0-9\s]', '', s)
    #remove prefix: h., x., t.
    s = re.sub(r'^(h\.|x\.|t\.)\s*', '', s)
    s = re.sub(r'\b(h|x|t)\b', ' ', s)
    #remove duplicate char, i.e, aabb => ab
    s = re.sub(r'(.)\1+', r'\1', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

# Building Trie for provinces, districts and wards

In [ ]:
import time
import sys

class TrieNode:
    def __init__(self):
      #Since the node is normalized, we should store the whole word instead is_terminal
      self.word = None
      self.children = {}

    def insert(self,word):
        node = self
        for letter in word:
            letter = strip_accents(letter).lower()
            if letter not in node.children:
                node.children[letter] = TrieNode()
            node = node.children[letter]
        node.word = word

# read dictionary file into a trie
provinceTrie = TrieNode()
for word in provinces:
    provinceTrie.insert(word)
districtTrie = TrieNode()
for word in districts:
    districtTrie.insert(word)
wardTrie = TrieNode()
for word in wards:
    wardTrie.insert(word)

# **Main function**
*Description*: Searching for a word using Trie and levanstein distance approach(i.e. edit distance).

*    While traversing the Trie, we also calculate the levanstein distance's row for the character (i.e. edit cost).
*    Once we meet the node which contain the word (`is_terminal = true/word != None`), we start compare the number of edit to the edit threshold:
1.   If it satisfied, we store that word along with the cost.
2.   If the word still have more childs and edit threshold is not break, we continue searching recursively.  
*   Finally, return the best word.

In [ ]:
class SearchResult:
    def __init__(self):
        self.word = ""
        self.cost = float('inf')

def findBestResult(word:str, trie:TrieNode, editThreshold:int, debug = False) -> SearchResult:
    #calculate basecase for levastein distance
    currentRow = range(len(word) + 1)
    result = SearchResult();
    # recursively search each branch of the trie
    for letter in trie.children:
        findAllPossibleResults(trie.children[letter],
                        letter,
                        word,
                        currentRow,
                        result,
                        editThreshold,
                        debug)
    return result

def findAllPossibleResults(node: TrieNode,
                    letter:str,
                    word:str,
                    previousRow:range,
                    result: SearchResult,
                    editThreshold: int,
                    debug = False):
    columns = len(word) + 1
    currentRow = [previousRow[0] + 1]

    #here we calculate each levanstein distance's row for the character
    for column in range(1, columns):
        insertCost = currentRow[column - 1] + 1
        deleteCost = previousRow[column] + 1
        if word[column - 1].lower() != letter:
            replaceCost = previousRow[column - 1] + 1
        else:
            replaceCost = previousRow[column - 1]
        currentRow.append(min(insertCost, deleteCost, replaceCost))

    #If the node is a complete word, edit cost is < limit and the cost is smallest, we store it
    if currentRow[-1] <= editThreshold and node.word != None:
        if debug:
            print(f"({node.word}-{currentRow[-1]})")
        if currentRow[-1] < result.cost:
            result.word = node.word
            result.cost = currentRow[-1]
    #After that, we continue searching for more possible results on the child if we are still able to (<= editThreshold)
    if min(currentRow) <= editThreshold:
        for letter in node.children:
            findAllPossibleResults(node.children[letter],
                            letter,
                            word,
                            currentRow,
                            result,
                            editThreshold, debug)

# Execution and evaluate the result

In [ ]:
class Result:
    def __init__(self):
        self.province = SearchResult()
        self.district = SearchResult()
        self.ward = SearchResult()

    def __str__(self):
        return f"Province: {self.province.word} | District: {self.district.word} | Ward: {self.ward.word}"

def tokenizeString(text, min_words=1, max_words=4):
    words = text.split()
    tokens = []

    for n in range(min_words, max_words + 1):
        for i in range(len(words) - n + 1):
            token = ' '.join(words[i:i+n])
            tokens.append(token)

    return tokens

def findBestMatchAddress(text: str):
    # build candidate substrings (window up to 4 tokens; addresses usually short).
    #References: https://github.com/phuochtran/address-classification/blob/main/gpt.md
    candidates = tokenizeString(text)
    finalResult = Result()
    for candidate in candidates:
        provinceResult = findBestResult(candidate, provinceTrie, 3)
        districtResult = findBestResult(candidate, districtTrie, 3)
        wardResult = findBestResult(candidate, wardTrie, 3)
        #Compare and get best result for each level
        if finalResult.province.cost > provinceResult.cost:
            finalResult.province = provinceResult
        if finalResult.district.cost > districtResult.cost:
            finalResult.district = districtResult
        if finalResult.ward.cost > wardResult.cost:
            finalResult.ward = wardResult

    return finalResult

In [ ]:
start = time.time()
examples = [
    "X. Thuận Thành, H. Cần Giuộc, T. Long Aaaaan",
    "Thuận Thanh, HCần Giuộc, Tlong An",
    "Thuận Thành, H Cần Giuộc T. Long An",
    "X ThuanThanh H. Can Giuoc, Long An",
    "ThuanThanh H Can Giuoc Long An",
    "X ThuanThanh H. Can Giuoc, LongAn",
    "H. Can   Giuoc, Long An",
    "ThuanThanh H Can Giuoc",
    "ThuanThanh Long An",
]
for example in examples:
    start = time.time()
    norm_text = normalize(example)
    print(f"{example} => {norm_text}")
    finalResult = findBestMatchAddress(norm_text)
    end = time.time()
    print(f"Evaluate {finalResult} within {end - start} s")

X. Thuận Thành, H. Cần Giuộc, T. Long Aaaaan => thuan thanh can giuoc long aaaaan
Evaluate Province: Long An | District: Cần Giuộc | Ward: Thuận Thành within 0.004898548126220703 s
Thuận Thanh, HCần Giuộc, Tlong An => thuan thanh hcan giuoc tlong an
Evaluate Province: Long An | District: Cần Giuộc | Ward: Thuận Thành within 0.005359172821044922 s
Thuận Thành, H Cần Giuộc T. Long An => thuan thanh can giuoc long an
Evaluate Province: Long An | District: Cần Giuộc | Ward: Thuận Thành within 0.005868196487426758 s
X ThuanThanh H. Can Giuoc, Long An => thuanthanh can giuoc long an
Evaluate Province: Long An | District: Cần Giuộc | Ward: Thuận Thành within 0.0034797191619873047 s
ThuanThanh H Can Giuoc Long An => thuanthanh can giuoc long an
Evaluate Province: Long An | District: Cần Giuộc | Ward: Thuận Thành within 0.004303455352783203 s
X ThuanThanh H. Can Giuoc, LongAn => thuanthanh can giuoc longan
Evaluate Province: Long An | District: Cần Giuộc | Ward: Thuận Thành within 0.00491476058